In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import nltk
from nltk.corpus import treebank
from nltk.corpus import brown

In [ ]:
# import seaborn as sns

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
# from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Input, Activation
from keras.layers import TimeDistributed
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, RNN
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.text import Tokenizer

In [ ]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [ ]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [ ]:
brown_corpus = brown.tagged_sents(tagset='universal')

In [ ]:
tagged_sentences = brown_corpus

In [ ]:
X_sentences = []
Y_tags = []
for item in tagged_sentences:
  X = []
  Y = []
  for e in item:
    X.append(e[0])
    Y.append(e[1])
  X_sentences.append(X)
  Y_tags.append(Y)

In [ ]:
X_sentences[1], Y_tags[1]

(['The',
  'jury',
  'further',
  'said',
  'in',
  'term-end',
  'presentments',
  'that',
  'the',
  'City',
  'Executive',
  'Committee',
  ',',
  'which',
  'had',
  'over-all',
  'charge',
  'of',
  'the',
  'election',
  ',',
  '``',
  'deserves',
  'the',
  'praise',
  'and',
  'thanks',
  'of',
  'the',
  'City',
  'of',
  'Atlanta',
  "''",
  'for',
  'the',
  'manner',
  'in',
  'which',
  'the',
  'election',
  'was',
  'conducted',
  '.'],
 ['DET',
  'NOUN',
  'ADV',
  'VERB',
  'ADP',
  'NOUN',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  'ADJ',
  'NOUN',
  '.',
  'DET',
  'VERB',
  'ADJ',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  '.',
  '.',
  'VERB',
  'DET',
  'NOUN',
  'CONJ',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'NOUN',
  '.',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'DET',
  'DET',
  'NOUN',
  'VERB',
  'VERB',
  '.'])

In [ ]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(X_sentences)
X_encoded = word_tokenizer.texts_to_sequences(X_sentences)

In [ ]:
tag_tokenizer = Tokenizer()
tag_tokenizer.fit_on_texts(Y_tags)
Y_encoded = tag_tokenizer.texts_to_sequences(Y_tags)

In [ ]:
from keras.utils import pad_sequences

In [ ]:
X_padded = pad_sequences(X_encoded, maxlen=150, padding="pre", truncating="post")
Y_padded = pad_sequences(Y_encoded, maxlen=150, padding="pre", truncating="post")

In [ ]:
X, Y = X_padded, Y_padded

In [ ]:
path = '/content/drive/MyDrive/GoogleNews-vectors-negative300.bin' #https://drive.google.com/file/d/1S-R94SvCoAAOQYzuLvh5OudF_vFfEl46/view?usp=shar #
word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

FileNotFoundError: ignored

In [ ]:

embedding_weights = np.zeros((len(word_tokenizer.word_index) + 1, 300))
word_dict = word_tokenizer.word_index

for word, index in word_dict.items():
    try:
        embedding_weights[index, :] = word_dict[word]
    except KeyError:
        pass

In [ ]:
Y = to_categorical(Y)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=4)

In [ ]:
print("TRAINING DATA")
print('Shape of input sequences: {}'.format(X_train.shape))
print('Shape of output sequences: {}'.format(Y_train.shape))
print("-"*50)
print("TESTING DATA")
print('Shape of input sequences: {}'.format(X_test.shape))
print('Shape of output sequences: {}'.format(Y_test.shape))

RNN fixed embeddings

In [ ]:
nn_model = Sequential()
nn_model.add(Embedding(input_dim= len(word_dict) + 1, output_dim=300,input_length=150,weights = [embedding_weights],trainable=  True ))


nn_model.add(Dense(256, input_dim=300))
nn_model.add(Activation('relu'))
nn_model.add(Dense(13, activation="softmax"))
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
nn_model.compile(loss =  'categorical_crossentropy',optimizer =  sgd,metrics   =  ['accuracy'])

In [ ]:
nn_model.summary()

In [ ]:
nn_training = nn_model.fit(X_train, Y_train, batch_size=128, epochs=10)

In [ ]:
print("Model Accuracy on test set = ", nn_model.evaluate(X_test,Y_test))

In [ ]:
def build_model():
  nn_model = Sequential()
  nn_model.add(Embedding(input_dim= len(word_dict) + 1, output_dim=300,input_length=150,weights = [embedding_weights],trainable=  True ))


  nn_model.add(Dense(256, input_dim=300))
  nn_model.add(Activation('relu'))
  nn_model.add(Dense(13, activation="softmax"))
  sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
  nn_model.compile(loss =  'categorical_crossentropy',optimizer =  sgd,metrics   =  ['accuracy'])
  return nn_model




In [ ]:
#doing kfold cross validation  X_train, X_test, Y_train, Y_test
import random
k=5
num_validation_sample=len(X_train)//k
#np.random.shuffle(data)

validation_score=[]
for fold in range(k):
  validation_data=X_train[num_validation_sample*fold:num_validation_sample*(fold+1)] # valiation data
  validation_target=Y_train[num_validation_sample*fold:num_validation_sample*(fold+1)]

  training_data=X_train[:num_validation_sample*fold]+X_train[num_validation_sample*(fold+1):] #remaining data
  training_target=Y_train[:num_validation_sample*fold]+Y_train[num_validation_sample*(fold+1):]

  model=build_model()
  model.fit(training_data, training_target, batch_size=128, epochs=10)
  #model.fit(training_data,training_target,epochs=20,batch_size=100)

  validation_score=model.evaluate(validation_data,validation_target)
  validation_score.append(validation_score)

validation_score=np.average(validation_score)

# model=get_model()
# model.trian(testing_data)
# test_score=model.evaluate(test_data)
